In [1]:
## Import the required libraries

from tkinter import *
from tkinter import ttk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure


Bad key "text.kerning_factor" on line 4 in
C:\Users\shriks44\Anaconda3_2019\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
## Read the data set here

alldf = pd.read_csv("E:\\Data Science\\The_Data_Incubator\\Motor_Vehicle_Collisions_-_Crashes.csv")


C:\Users\shriks44\Anaconda3_2019\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
Detailed_project_summary = """ Motivation:

1. Vehicle crashes occur all over the world, with varying severity, traffic rules, vehicle characteristics, driving behavior, weather etc., each of which play a significant role in the outcome of the crash. 

2. While most of the data science applications for crash analysis have been only in advanced visualization, there is limited study of developing advanced machine learning algorithms that can help predict future crashes and its severities based on several parameters.

3. The limited existing models are developed for much larger areas which may result in overfit/underfit issue. Hence, 'One model fits all' is an outdated strategy and will not be beneficial in providing actionable insights to zone specific crashes which may have different parameters. 

Project at The Data Incubator:

1. Through my project at the Data Incubator, I wish to develop an automated system which in return develops machine learning models to predict future fatal or non fatal crashes for any chosen area. 

2. These models would be focussed towards very specific zones/areas/road corridors, which I term as micro-crash models.

3. The intention here is to develop custom models that are built solely on the data that is represented by that zone/area rather than a much larger set. 

4. Having such area specific models through an automated system will be beneficial to a lot of state or federal Department's of Transport who can have access to a tool that can deliver key actionable insights, and help direct their crash preventive measures more effectively in future.

5. This can help direct their budget, initiatives, and efforts to specific locations that need the most attention. Outcomes of such models can be imposing traffic regulations during peak crash hours, zones, restricitng certain category of vehicles etc.

Data availability:

1. A lot of Transportation agencies, City DOT's, Federal DOT's provide traffic and crash related open data that can be easily acquired to develop this bigger idea. 

2. I wish to develop a full fledged application that can be responsive in real time and update the models with live feed of additional data through pipelines whenever available. As of today, there is no such automated model development application in Transportation.
"""

In [6]:
import tkinter as tk

root = tk.Tk()
root.title("Welcome to New York City Micro-Crash modeling")
root.geometry("1000x600")

#alldf = pd.read_csv("E:\\Data Science\\The_Data_Incubator\\Motor_Vehicle_Collisions_-_Crashes.csv")

option_1 = ['Select', 'ZIP CODE', 'BOROUGH']

option_2 = list(alldf['BOROUGH'].unique())
option_2 = [x for x in option_2 if str(x) != 'nan']
option_2.insert(0, 'Select')


#################### user_entry_frame ###########################
user_entry_frame = tk.Frame(root, background="light blue", borderwidth = 1,highlightthickness=3)
user_entry_frame.place(x=0, y=0, anchor="nw", width=450, height=300)

user_label_1 = Label(user_entry_frame, text="Model development inputs", bg="sky blue")
user_label_1.grid(row=0, column=0, padx=1, pady= 1)

user_label_2 = Label(user_entry_frame, text="Select if you would like a model based on Zip Codes or Borough")
user_label_2.grid(row=2, column=0, padx= 15, pady= 10)

myCombo1 = ttk.Combobox(user_entry_frame, value=option_1)
myCombo1.current(0)
#myCombo1.bind("<<ComboboxSelected>>", selected)
myCombo1.grid(row=4, column=0, padx = 0, pady= 10)



#### Model selection

def model_prediction():
    
    import warnings
    warnings.filterwarnings("ignore")
    criteria_1 = myCombo1.get()
    zone_id = myCombo2.get()
    df = alldf[alldf[criteria_1] == zone_id]
    
    ## We will try to model total fatalities that occur for a particular incident
    ## In our modelling case, we would like to have strong predictors. We can obtain few indicators from the date column
    ## We can split the date column and obtain crucial information such as month and weekday/weekend which may provide crucial insights
    df['month'] = pd.DatetimeIndex(df['CRASH DATE']).month
    df['pd_datetime'] = pd.to_datetime(df['CRASH DATE'])
    df['weekday'] = df['pd_datetime'].dt.dayofweek
    df['hour'] = pd.to_datetime(df['CRASH TIME']).dt.hour
    
    ## We can see that number of fatalities is a continuous random variable. Hence we will categorize that data into fatal(1) or non-fatal(0)
    df['output'] = df['NUMBER OF PERSONS KILLED'].apply(lambda x: 1 if x>0 else 0)

    ## Similarly, we will make other predictors as categorical.
    ## Based on initial screening, will use our sample predictors as weekday, hour, month, 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
    #'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
    ## We will convert the non-numeric data into numeric by usink a lebel-encoder

    from sklearn.preprocessing import LabelEncoder

    le_factor = LabelEncoder()
    uniquelist_factor = df['CONTRIBUTING FACTOR VEHICLE 1'].unique().tolist() + df['CONTRIBUTING FACTOR VEHICLE 2'].unique().tolist() + df['CONTRIBUTING FACTOR VEHICLE 3'].unique().tolist() + df['CONTRIBUTING FACTOR VEHICLE 4'].unique().tolist() + df['CONTRIBUTING FACTOR VEHICLE 5'].unique().tolist()
    uniquelist_factor = list(dict.fromkeys(uniquelist_factor))
    le_factor.fit(uniquelist_factor)
    df['CONTRIBUTING FACTOR VEHICLE 1'] = le_factor.transform(df['CONTRIBUTING FACTOR VEHICLE 1'].astype(str))
    df['CONTRIBUTING FACTOR VEHICLE 2'] = le_factor.transform(df['CONTRIBUTING FACTOR VEHICLE 2'].astype(str))
    df['CONTRIBUTING FACTOR VEHICLE 3'] = le_factor.transform(df['CONTRIBUTING FACTOR VEHICLE 3'].astype(str))
    df['CONTRIBUTING FACTOR VEHICLE 4'] = le_factor.transform(df['CONTRIBUTING FACTOR VEHICLE 4'].astype(str))
    df['CONTRIBUTING FACTOR VEHICLE 5'] = le_factor.transform(df['CONTRIBUTING FACTOR VEHICLE 5'].astype(str))
    
    
    # We can see that there are only a few fatal cases as compared to non-fatal cases.
    # Hence we will select the data to have equal values of both
    least_samples = df.groupby('output')['hour'].count().min()
    nonfatal_df = df[df['output'] == 0].sample(n=least_samples)
    fatal_df = df[df['output'] == 1]
    combined = [nonfatal_df, fatal_df]
    final_output_df = pd.concat(combined)
    
    
    ## Now we will define our input and output variables for developing the model
    trainset = final_output_df[['weekday','month','hour','CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']]
    labels  = final_output_df['output']
    
    ## To enhance reliability of the accuracy score, we will use different folds of the data for training and testing
    
    from sklearn.model_selection import StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(10, test_size = 0.25, random_state = 23)

    sss.get_n_splits(trainset, labels)
    train_x, test_x, train_y, test_y = [],[],[],[]

    for train_index, test_index in sss.split(trainset, labels):
        train_x.append(trainset.iloc[train_index])
        test_x.append(trainset.iloc[test_index])
        train_y.append(labels.iloc[train_index])
        test_y.append(labels.iloc[test_index])
        
    ### We will use 7 diffeent classifiers to test the model accuracy
     
    from sklearn import metrics as sm
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC, LinearSVC, NuSVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import AdaBoostClassifier

    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel='rbf', C=0.025, probability=True, gamma='scale'),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        GaussianNB(),
        LogisticRegression(),
        AdaBoostClassifier(n_estimators = 30)
    ]
    
    
    def Average(lst): 
        return sum(lst) / len(lst)
    
    from prettytable import PrettyTable
    Results_table = PrettyTable(['Classifier', 'Accuracy'])

    classifier_name = []
    classifier_accuracy = []

    for clf in classifiers:
        acc_total=[]
        for i in range(len(train_x)):
        
            clf.fit(train_x[i], train_y[i])
            #name = type(clf).__name__
            model_name = clf.__class__.__name__

            train_predictions = clf.predict(test_x[i])
            acc = sm.accuracy_score(test_y[i] , train_predictions)
            acc_total.append(acc)
        Results_table.add_row([str(model_name), "{:.2%}".format(Average(acc_total))])
        classifier_name.append(str(model_name))
        classifier_accuracy.append(Average(acc_total)*100)
    
    ### Here we will develop a plot to show summary of classifier accuracy
    
    fig_model_output = Figure(figsize=(5.4,2.9), dpi=100)
    ax1 = fig_model_output.add_subplot(111)
    ax1.barh(classifier_name,classifier_accuracy)
    ax1.set_xlabel('Accuracy (%)')
    ax1.set_ylabel('Classifier') 
    ax1.set_title('Model summary for: ' + zone_id)
    ax1.set_xticks(np.arange(0,100,10))
    ax1.xaxis.set_tick_params(labelsize=8)
    ax1.yaxis.set_tick_params(labelsize=8)
    fig_model_output.tight_layout()
    
    canvas_model_output = FigureCanvasTkAgg(fig_model_output, master=model_output_frame)
    canvas_model_output.get_tk_widget().grid(row=0, column=0)
    canvas_model_output.draw()
    
    ### Here we will develop a plot to show fatalities by hour for the selected zone
    
    chart_data = df.groupby('hour')['NUMBER OF PERSONS KILLED'].sum().reset_index()
    fig_statistics = Figure(figsize=(5.4,2.9), dpi=100)
    ax2 = fig_statistics.add_subplot(111)
    ax2.bar(chart_data['hour'], chart_data['NUMBER OF PERSONS KILLED'], color='g')
    ax2.set_xlabel('Hour')
    ax2.set_ylabel('Total fatalities') 
    ax2.set_title('Total fatalities by hour for : ' + zone_id)
    ax2.set_xticks(np.arange(len(chart_data['hour'])))
    ax2.xaxis.set_tick_params(labelsize=8)
    ax2.yaxis.set_tick_params(labelsize=8)
    fig_statistics.tight_layout()

    canvas_statistics = FigureCanvasTkAgg(fig_statistics, master=zone_summary_frame)
    canvas_statistics.get_tk_widget().grid(row=0, column=0)
    canvas_statistics.draw()

    
### Model selection done
    
l2 = Label(user_entry_frame, text="Select the corresponding Zip Code or Borough")
l2.grid(row=6, column=0, padx=0, pady= 10)


myCombo2 = ttk.Combobox(user_entry_frame, value = option_2)
myCombo2.current(0)
#myCombo2.bind("<<ComboboxSelected>>", model_prediction)
myCombo2.grid(row=8, column=0, padx=0, pady= 10)

generate_model_button = tk.Button(user_entry_frame, text="Click here to generate model and results", command = model_prediction)
generate_model_button.grid(row=10, column=0, padx=10, pady= 40)


#################### project_summary_frame ###########################
project_summary_frame = tk.Frame(root, background="light blue", borderwidth = 1,highlightthickness=3)
project_summary_frame.place(x=0, y=300, anchor="nw", width=450, height=300)


l1 = Label(project_summary_frame, bg="sky blue")
l1.grid(row=0, column=0, padx=1, pady= 1)


def show_project_summary ():
    global shown
    if shown: project_summary.grid_remove () # Hide the text
    else: project_summary.grid(row=2, column=0, padx=2, pady=5) # Show the text
    shown = not shown # Reverse the 'shown' boolean value
    
    
project_summary = Text(project_summary_frame, height=15, width=54, wrap= WORD)#, yscrollcommand = scrollbar.set)
#project_summary.grid(row=1, column=0)
project_summary.insert(INSERT, Detailed_project_summary)

shown = False
generate_summary_button = tk.Button(project_summary_frame, text="Kindly click here for project summary", command = show_project_summary, bg="sky blue")
generate_summary_button.grid(row=0, column=0, padx = 0, pady = 2)
                           

#################### model_output_frame ###########################
model_output_frame = tk.Frame(root, background="light blue", borderwidth = 1,highlightthickness=3)
model_output_frame.place(x=450, y=0, anchor="nw", width=550, height=300)


l1 = Label(model_output_frame, text="Model performance charts", bg="sky blue")
l1.grid(row=0, column=0, padx=1, pady= 1)

#chart = FigureCanvasTkAgg(fig, model_output_frame)
#chart.get_tk_widget().grid(row=0, column=0, padx=1, pady= 1)

#################### zone_summary_frame ###########################
zone_summary_frame = tk.Frame(root, background="light blue", borderwidth = 1,highlightthickness=3)
zone_summary_frame.place(x=450, y=300, anchor="nw", width=550, height=300)

l1 = Label(zone_summary_frame, text="Zone summary charts", bg="sky blue")
l1.grid(row=0, column=0, padx=1, pady= 1)


root.mainloop()